https://plotnine.readthedocs.io/en/stable/api.html

## Session prep

In [ ]:
#Import packages
import pandas as pd
from  plotnine import *

In [ ]:
#Instruct Jupyter to display plots inline
%matplotlib inline

## 1. Loading the data into Python

In [ ]:
#Read the WaterUse and Population worksheets into data frames
dfWaterUse = pd.read_excel('../data/State_Data_Formatted.xlsx','WaterUse')
dfPopulation = pd.read_excel('../data/State_Data_Formatted.xlsx','Population')

## 2. Joining tables

In [ ]:
#Join the two dataframes into one using the Pandas "merge" command
dfAll = pd.merge(left=dfWaterUse,     #Specify the "left" table
                 right=dfPopulation,  #Specify the "right" table
                 how='inner',         #Specify the type of join (vs "outer", "left", or "right")
                 left_on = 'State',   #Specify the join field in the "left" table
                 right_on = 'STATE'   #Specify the join field in the "right" table
                 )
dfAll.head()

## 3. Construct the visualization
Recall the rotated stacked bar plot of water withdrawal by type and category we created in Tableau:
![Plot1](https://datadevils.github.io/DataBootCamp/media/tableau/BarChart3.png)
This is what we want to recreate using `ggplot` via the Python "plotnine" package. 

### 3a. Organizing our data
In Tableau, we had to organize our data. Here, our data exist in a tidy format (one and only one row for each observation, with a column for each property of that observation). `ggplot` is designed to work with data in that format, so no need to further organize our data! We do, however, need to add a "Per capita water use" field. 

In [ ]:
#Compute MGD per capita field
dfAll['MGD_perCapita'] = dfAll['Withdrawal_MGD'] / dfAll['Population']
dfAll.head()

### 3b. Plotting our data
The next step is to construct our plots, which we will be doing with the `Plotnine` package. This package enables us to use the`ggplot` interface, which has a bit of a steep learning curve. So, let's pause and introduce the grammer of graphics...

---
### ►`ggplot` _and the Grammer of Graphics_ ◄
Plots have numerous options in presentation, design, and content. The Grammer of Graphics ("GG") is a set of structural rules defining the components of a plotting language that strives to simplify designing plots from such a wide array of options from a coding platform. Even so, it has a bit of a steep learning curve, so to help with that, let's examine it from it's foundations. 

The ["CheatSheet"](https://www.rstudio.com/wp-content/uploads/2016/11/ggplot2-cheatsheet-2.1.pdf) for `ggplot2` (the "R" version of what we are doing) provides a quick introduction to GG. In it, we see that plots are divided into a set of components that our code must specify:
* First is the source of the **data** for our plot. 
* Associated with the data object are its **aesthetics**. The aesthetics include the columns that we wish to plot, as well as the size and color attributes we want to assign to these features. 
* Then we specify a **`geom`** object, which indicate how we want to plot our data. We can specify multiple `geom` objects.

There are some more elements in the grammer of graphics, but let's start there and experiment. First, let's see if we can replicate the stacked bar plot we did in Tableau.  

---

### Stacked bar plot
Take a look at the ggplot2 cheatsheet. Specifically, look at the various graphical primitives, or "**geoms**", we have at our disposal. They are organized by the types of data we wish to plot. In our example, we want to plot one discrete value ("`Category`") and one continuous one ("`MGD_perCapita`"). The option for `geom_col` appears to serve our purpose. Let's try that: 

This is a two step process. First we create the base ggplot object, telling it to use the `dfAll` dataframe as the source of data. Then, we add our `geom_col` object, using its `aes` object to define which columns to use in the plot. 

In [ ]:
#Create the ggplot object, defining it to use our dfAll dataframe as the source of data
myPlot = ggplot(data=dfAll)

In [ ]:
#Now add the geom_col object to our ggplot object along with some aesthetics, and it will plot
myPlot + geom_col(aes(x='Category', y='MGD_perCapita'))

Not perfect, but a good start. To learn what we just did, making a plot showing total withdrawal by source vs by category. 

#### Aesthetics: *fill*
Next, we can modify our plot by tweaking the aesthetics. Here we'll use the `fill` property, which colors the bar based on the property we specify. (I've also now split the command across mulitple lines to improve readability.)

In [ ]:
#Now add the geom_col object
myPlot + geom_col(aes(x='Category',
                      y='MGD_perCapita',  
                      fill='Source',
                     ))

#### Facets: *facet_grid*
More interesting and closer to our ultimate goal, but we also want separate plots for Fresh vs Saline water withdrawals. This is done using **facets**. Facets split data on values in one or more fields, creating a separate plot for each subset of data. Below the `facet_grid` command causes our plot to create separate plots based on values the `Type` field. (*I've again modified the format of the Python command to make it more readable...*)

The `facets='.~Type'` argument passed to the facet_grid indicated that we want to create a new plot for each unique value in the "Type" column (i.e. "Fresh" or "Saline"), and place those side by side. Try changing the value to `Type~.` and see what happens. Then try `Source~Type`. This reveals that we can specify various fields to represent rows and columns of plot facets from our dataset; the field written before the `~` is used for rows of plots and after the `~` for columns. And we can add a `.` on either side as a placeholder if we only want one field. 

In [ ]:
#Create facets based on the Type column
(ggplot(data=dfAll) + 
 geom_col(aes(x='Category',
              y='MGD_perCapita',
              fill='Source')) + 
 facet_grid(facets='.~Type'))     #Adds a seperate "facet" for Fresh and Saline types; plots side by side

#### Labels and coordinate systems
Closer to what we want, but we still need to:
* add a title to our plot,
* tidy up the axis labels, and
* arrange the data so the bars are horizontal
The first two can be done using label commands found in the cheat sheet (see the `Labels` section), and the command for swapping axes is found in the `coordinate systems` section. Notice the bottom three additions to the code used to create our plot. Also notice how we modify the argument supplied to the `facet_grid` command: by switching from `.~Type` to `Type~.` (see above).

In [ ]:
#Add/fix labels, swap axes, and place facets horizontally
(ggplot(data=dfAll) + 
 geom_col(aes(x='Category',
              y='MGD_perCapita',
              fill='Source')) + 
 facet_grid(facets='Type ~ .') +
 ggtitle('Per capita water withdrawals') + 
 ylab("Per capita water withdrawals (MGD/person)") +
 coord_flip() 
)

To complete our plot, we still need to remove the Total category, show results for Fresh water withdrawals only, and sort the values in our plot by category, showing them from least to largest per capita withdrawal.

In [ ]:
#First we create a dataframe without the "Total" category and with only the "Fresh" type
dfSubset = dfAll.query('Category != "Total" and Type == "Fresh"')

In [ ]:
#Create a list of Category values sorted by MGD_PerCapita
dfSorted = (dfSubset.groupby('Category')              #Group data on Category
                    .agg({'MGD_perCapita':'sum'})     #Compute the sum of the MGD_perCapita for each Category value
                    .sort_values(by='MGD_perCapita',  #Sort values based on sum MGD_perCapita
                                 ascending=False))     #in descending order
#Show the resulting dataframe
dfSorted

In [ ]:
#Export the index of the sorted dataframe to a Python list
Category_list = dfSorted.index.tolist()               
print(Category_list)

In [ ]:
#Convert these values to a Pandas Categorical object and add back to the original dataframe
Category_sort = pd.Categorical(dfSubset['Category'],categories=Category_list)

#Assign these values to a new column in the dataframe
dfSubset = dfSubset.assign(Category_sort = Category_sort)

In [ ]:
#Create facets based on the Type column
(ggplot(data=dfSubset) + 
 geom_col(aes(x='Category_sort', #Note: X field now set to the Category_sort column
              y='MGD_perCapita',
              fill='Source')) + 
 #facet_grid(facets='Type ~ .') + #Comment out facet command as we now only have Fresh
 ggtitle('Per capita water withdrawals') + 
 ylab("Per capita water withdrawals (MGD/person)") +
 xlab("Category") +              #X label added 
 coord_flip() 
)

### 3c. Filter for just Fresh

In [ ]:
df_Fresh = dfAll.query('Type == "Fresh"')
df_Fresh.index = 
(ggplot(data=df_Fresh[dfAll.Category != 'Total'],
        mapping=aes(x='Category',y='Withdrawal_MGD',fill='Source')) +
  geom_col() +
  coord_flip() +
  #facet_wrap(facets = 'Type',ncol=1) + 
  ylab('Total withdrawal (MGD)') +
  theme(axis_text = element_text(angle=0)) 
)